In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import entropy
import numpy as np

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out = [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        if n_sample == 0:
            n_sample = 1

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0)


def classwise_uncertainty_sampling_by_fraction(X_pool, y_pool, model, fraction=0.2, random_state=42):
    X_selected = []
    y_selected = []
    selected_indices = []

    classes = np.unique(y_pool)
    probs = model.predict_proba(X_pool)
    rng = np.random.RandomState(seed=random_state)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        X_c = X_pool[idx_c]
        probs_c = probs[idx_c]


        n_c = len(idx_c)
        n_select = max(1, int(n_c * fraction))

        if n_c == 0:
            continue

        uncertainties = entropy(probs_c.T)  # shape: (n_samples_in_class,)
        sorted_idx = np.argsort(uncertainties)[-n_select:]  # top-k uncertain

        top_k_idx = idx_c[sorted_idx]

        selected_indices.extend(top_k_idx)
        X_selected.append(X_pool[top_k_idx])
        y_selected.append(y_pool[top_k_idx])

    selected_indices = np.array(selected_indices)
    X_selected = np.concatenate(X_selected, axis=0)
    y_selected = np.concatenate(y_selected, axis=0)

    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    return X_rest, X_selected, y_rest, y_selected

X_pool = x_test_adv_0_1 
y_pool = y_label_0_1
X_train_init, y_train_init = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)


_, X_50, _, y_50 = classwise_uncertainty_sampling_by_fraction(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.5
)



In [6]:
X_50.shape

(1074385, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb50 = XGBClassifier()
xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt50 = DecisionTreeClassifier()
# dt50.fit(X_50, y_50)

xgb
RF


RandomForestClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 2 4 5 7] [     3      2      1 238425    322]
(3, 42) (3,)
Save 1 to baseline_Def2.npy
(2, 42) (2,)
Save 2 to baseline_Def3.npy
(1, 42) (1,)
Save 4 to baseline_Def5.npy
(238425, 42) (238425,)
Save 5 to baseline_Def6.npy
(322, 42) (322,)
Save 7 to baseline_Def8.npy
Execution Time: 1.701825 seconds
(716259,)
[0 2 3 4 5 7 9] [252293 278343      7    662 184163    777     14]
(252293, 42) (252293,)
Save 0 to BIM_Def1.npy
(278343, 42) (278343,)
Save 2 to BIM_Def3.npy
(7, 42) (7,)
Save 3 to BIM_Def4.npy
(662, 42) (662,)
Save 4 to BIM_Def5.npy
(184163, 42) (184163,)
Save 5 to BIM_Def6.npy
(777, 42) (777,)
Save 7 to BIM_Def8.npy
(14, 42) (14,)
Save 9 to BIM_Def10.npy
Execution Time: 3.651925 seconds
(716259,)
[0 1 2 3 4 5 7 8 9] [447943      1 244430      1  13067   9534    969      1    313]
(447943, 42) (447943,)
Save 0 to FGSM_Def1.npy
(1, 42) (1,)
Save 1 to FGSM_Def2.npy
(244430, 42) (244430,)
Save 2 to FGSM_Def3.npy
(1, 42) (1,)
Save 3 to FGSM_Def4.npy
(13067, 42) (13067,)
Sa

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 2 3 4 5 7] [     2      8      1      1 237651   1090]
(2, 42) (2,)
Save 1 to baseline_Def2.npy
(8, 42) (8,)
Save 2 to baseline_Def3.npy
(1, 42) (1,)
Save 3 to baseline_Def4.npy
(1, 42) (1,)
Save 4 to baseline_Def5.npy
(237651, 42) (237651,)
Save 5 to baseline_Def6.npy
(1090, 42) (1090,)
Save 7 to baseline_Def8.npy
Execution Time: 24.968310 seconds
(716259,)
[0 2 4 5 7 9] [224500 487828   1233   1057   1405    236]
(224500, 42) (224500,)
Save 0 to BIM_Def1.npy
(487828, 42) (487828,)
Save 2 to BIM_Def3.npy
(1233, 42) (1233,)
Save 4 to BIM_Def5.npy
(1057, 42) (1057,)
Save 5 to BIM_Def6.npy
(1405, 42) (1405,)
Save 7 to BIM_Def8.npy
(236, 42) (236,)
Save 9 to BIM_Def10.npy
Execution Time: 9.305791 seconds
(716259,)
[0 1 2 4 5 7 9] [439689      2 250323   5945  17361   1871   1068]
(439689, 42) (439689,)
Save 0 to FGSM_Def1.npy
(2, 42) (2,)
Save 1 to FGSM_Def2.npy
(250323, 42) (250323,)
Save 2 to FGSM_Def3.npy
(5945, 42) (5945,)
Save 4 to FGSM_Def5.npy
(17361, 42) (17361,)
Save

In [10]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt50.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_ActiveLearning/WUSTL_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50_ActiveLearning.csv")

In [13]:
all_attack_list

0       1   2       3      4      5       6   7       8     9      10  \
0   XGB50       0   3       2      0      1  238425   0     322     0      0   
1   XGB50  252293   0  278343      7    662  184163   0     777     0     14   
2   XGB50  447943   1  244430      1  13067    9534   0     969     1    313   
3   XGB50  252293   0  278343      7    662  184163   0     777     0     14   
4   XGB50     111   0    6433     30  10007  688347   5    2828     1   8497   
5   XGB50   90938   3  579181     39    493   43765   0    1751     0     89   
6   XGB50       0   7      19      0      4  715250   0     978     1      0   
7   XGB50    8442   0    4026  42120   2273   61256   0  596814  1174    148   
8   XGB50    3459   0  710215      0     38    2454   0       0     0     93   
9   XGB50  422503   0  251576      6   3168   38476   0     516     0     14   
10   RF50       0   2       8      1      1  237651   0    1090     0      0   
11   RF50  224500   0  487828      0   1233    1057   0    1405     0    236   
12   RF50  439689   2  250323      0   5945   17361   0    1871     0   1068   
13   RF50  224500   0  487828      0   1233    1057   0    1405     0    236   
14   RF50      75   0    6007     34   8560  688262   0    2987     6  10328   
15   RF50  123761   0  485967      0     95  105093   0    1341     0      2   
16   RF50       0   6      22      3      3  712941   0    3284     0      0   
17   RF50    3885   0    3043  15143   1334   56221   0  636269   332     32   
18   RF50   23041   0  690850      0      0    2368   0       0     0      0   
19   RF50  377983   0  263961      0   1345   72589   0     356     0     25   
20   DT50       0   0       0      0      0       0   0       0     0      0   
21   DT50       0   0       0      0      0       0   0       0     0      0   
22   DT50       0   0       0      0      0       0   0       0     0      0   
23   DT50       0   0       0      0      0       0   0       0     0      0   
24   DT50       0   0       0      0      0       0   0       0     0      0   
25   DT50       0   0       0      0      0       0   0       0     0      0   
26   DT50       0   0       0      0      0       0   0       0     0      0   
27   DT50       0   0       0      0      0       0   0       0     0      0   
28   DT50       0   0       0      0      0       0   0       0     0      0   
29   DT50       0   0       0      0      0       0   0       0     0      0   

    11      12  
0    0  238753  
1    0  716259  
2    0  716259  
3    0  716259  
4    0  716259  
5    0  716259  
6    0  716259  
7    6  716259  
8    0  716259  
9    0  716259  
10   0  238753  
11   0  716259  
12   0  716259  
13   0  716259  
14   0  716259  
15   0  716259  
16   0  716259  
17   0  716259  
18   0  716259  
19   0  716259  
20   0       0  
21   0       0  
22   0       0  
23   0       0  
24   0       0  
25   0       0  
26   0       0  
27   0       0  
28   0       0  
29   0       0